In [16]:
import pandas as pd 
import numpy as np

In [17]:
df=pd.read_csv('D:\Food Delivery Time Prediction\Data\RawData\Food_Delivery_Times.csv')

X=df.drop(columns=['Delivery_Time_min','Order_ID','Courier_Experience_yrs'],axis=1)
y=df['Delivery_Time_min']


In [18]:
X.columns

Index(['Distance_km', 'Weather', 'Traffic_Level', 'Time_of_Day',
       'Vehicle_Type', 'Preparation_Time_min'],
      dtype='object')

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import root_mean_squared_error,mean_absolute_error,r2_score

In [20]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(include=['object']).columns

In [21]:
ordinal_cat=['Traffic_Level','Time_of_Day']
onehot_cat=['Weather','Vehicle_Type']
traffic_mapping = ['Low', 'Medium', 'High']
time_mapping = ['Afternoon', 'Evening', 'Night', 'Morning']

In [22]:
numerical_pipeline=Pipeline(
    steps=[
        ('Imputer',SimpleImputer(strategy='mean')),
        ('Scaling',StandardScaler())
    ]
)
cat_oh_pipeline=Pipeline(
    steps=[
        ('Imputer',SimpleImputer(strategy='most_frequent')),
        ('Onehot',OneHotEncoder(drop='first'))
    ]
)
cat_ordinal_pipeline=Pipeline(
    steps=[
        ('Imputer',SimpleImputer(strategy='most_frequent')),
        ('Ordinal',OrdinalEncoder(categories=[traffic_mapping,time_mapping]))
    ]
)

In [23]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num_features',numerical_pipeline,num_features),
        ('onehot',cat_oh_pipeline,onehot_cat),
        ('ordinal',cat_ordinal_pipeline,ordinal_cat)
    ]
)

In [24]:
X=preprocessor.fit_transform(X)

In [25]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,random_state=42)

In [26]:
models = {
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'XGBoost': XGBRegressor(),
    'CatBoost': CatBoostRegressor(verbose=0)
}

In [27]:
def evaluate_model(y_train,y_pred):
    return mean_absolute_error(y_train,y_pred),root_mean_squared_error(y_train,y_pred),r2_score(y_train,y_pred)

In [28]:

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')

LinearRegression
Model performance for Training set
- Root Mean Squared Error: 11.1705
- Mean Absolute Error: 6.9432
- R2 Score: 0.7398
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 9.3160
- Mean Absolute Error: 6.2392
- R2 Score: 0.8267


DecisionTree
Model performance for Training set
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 13.7046
- Mean Absolute Error: 9.3100
- R2 Score: 0.6249


RandomForest
Model performance for Training set
- Root Mean Squared Error: 4.8027
- Mean Absolute Error: 3.1279
- R2 Score: 0.9519
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 10.2370
- Mean Absolute Error: 7.0180
- R2 Score: 0.7907


AdaBoost
Model performance for Training set
- Root Mean Squared Error: 14.3346
- Mean Absolute Error: 12.2862
- R2 Score: 0.5715
------------------

In [29]:
param_grids = {
    "LinearRegression": {
        "fit_intercept": [True, False],
        "positive": [True, False]
    },
    
    "DecisionTree": {
        "max_depth": [None, 5, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": [None, "sqrt", "log2"]
    },
    
    "RandomForest": {
        "n_estimators": [100, 200, 300],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["sqrt", "log2"]
    },
    
    "AdaBoost": {
        "n_estimators": [50, 100, 200, 300],
        "learning_rate": [0.001, 0.01, 0.1, 0.5, 1]
    },
    
    "GradientBoosting": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.001, 0.01, 0.1, 0.2],
        "max_depth": [3, 5, 7],
       
    },
    
    "XGBoost": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.001, 0.01, 0.1, 0.2],
        "max_depth": [3, 5, 7, 10],

    },
    
    "CatBoost": {
    "depth": [4, 6, 8, 10, 12],
    "learning_rate": [0.01, 0.03, 0.05, 0.1, 0.2],
    "iterations": [200, 500, 800, 1000],
    "l2_leaf_reg": [1, 3, 5, 7, 9],                # L2 regularization
    "bagging_temperature": [0, 0.5, 1, 2, 5],      # Controls sampling, like subsample          
    "random_strength": [0, 1, 2, 5, 10],           # Adds randomness to splits
    "rsm": [0.6, 0.8, 1.0],                        # Random subspace method (feature fraction)
    "grow_policy": ["SymmetricTree", "Depthwise", "Lossguide"], # Tree growth strategy
    "min_data_in_leaf": [1, 5, 10, 20, 50],        # Minimum samples per leaf
    "max_bin": [128, 254, 512]                     # Binning for numerical features
}

}


In [30]:
# Running tuning for all models
best_models = {}
for name, model in models.items():
    print(f"Tuning {name}...")
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grids[name],
        n_iter=20,              # number of combinations to try
        cv=5,                   # 5-fold cross-validation
        scoring="r2",
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    search.fit(X_train, y_train)
    best_models[name] = search.best_estimator_
    print(f"Best Params for {name}: {search.best_params_}")
    print(f"Best CV R²: {search.best_score_}")
    print("-"*50)


Tuning LinearRegression...
Fitting 5 folds for each of 4 candidates, totalling 20 fits


d:\Food Delivery Time Prediction\venv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params for LinearRegression: {'positive': True, 'fit_intercept': True}
Best CV R²: 0.7327411827197714
--------------------------------------------------
Tuning DecisionTree...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Params for DecisionTree: {'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 5}
Best CV R²: 0.5870721451921327
--------------------------------------------------
Tuning RandomForest...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Params for RandomForest: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}
Best CV R²: 0.6926407347856488
--------------------------------------------------
Tuning AdaBoost...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Params for AdaBoost: {'n_estimators': 100, 'learning_rate': 0.01}
Best CV R²: 0.6408033987407233
--------------------------------------------------
Tuning GradientB

## Out of all these models Linear Regression is better but for more accuracy for non linear data i will choose CatBoost to train my model

In [31]:

params= {
    "depth": [7, 8, 9],
    "learning_rate": [0.02, 0.03, 0.04],
    "iterations": [300, 500, 800],   # try longer training
    "l2_leaf_reg": [5, 6, 7, 8],
    "bagging_temperature": [0.3, 0.5, 0.7],
    "random_strength": [1, 2, 3],
    "min_data_in_leaf": [5, 10, 20],
    "rsm": [0.7, 0.8, 0.9],
    "max_bin": [254, 512],
    "grow_policy": ["SymmetricTree", "Lossguide"]
}

In [32]:
model=CatBoostRegressor()
rcv=RandomizedSearchCV(model,params,cv=5,n_jobs=-1)
rcv.fit(X_train,y_train)
y_pred=rcv.predict(X_test)
print(r2_score(y_test,y_pred))

0:	learn: 21.6769348	total: 5.45ms	remaining: 1.63s
1:	learn: 21.4390058	total: 11.3ms	remaining: 1.69s
2:	learn: 21.2058145	total: 17.4ms	remaining: 1.72s
3:	learn: 20.9672692	total: 22.9ms	remaining: 1.69s
4:	learn: 20.7590137	total: 29.6ms	remaining: 1.75s
5:	learn: 20.5588556	total: 34.8ms	remaining: 1.7s
6:	learn: 20.3630666	total: 40.5ms	remaining: 1.7s
7:	learn: 20.1580041	total: 47.6ms	remaining: 1.74s
8:	learn: 19.9511624	total: 52.9ms	remaining: 1.71s
9:	learn: 19.7736450	total: 58.5ms	remaining: 1.7s
10:	learn: 19.5900279	total: 64.2ms	remaining: 1.69s
11:	learn: 19.4061049	total: 70.7ms	remaining: 1.7s
12:	learn: 19.2243271	total: 77.1ms	remaining: 1.7s
13:	learn: 19.0752166	total: 82.3ms	remaining: 1.68s
14:	learn: 18.9228630	total: 88.1ms	remaining: 1.67s
15:	learn: 18.7555218	total: 95.1ms	remaining: 1.69s
16:	learn: 18.5971885	total: 101ms	remaining: 1.67s
17:	learn: 18.4469856	total: 107ms	remaining: 1.67s
18:	learn: 18.2841190	total: 113ms	remaining: 1.67s
19:	learn: 

In [33]:
rcv.best_params_

{'rsm': 0.7,
 'random_strength': 1,
 'min_data_in_leaf': 10,
 'max_bin': 512,
 'learning_rate': 0.02,
 'l2_leaf_reg': 5,
 'iterations': 300,
 'grow_policy': 'Lossguide',
 'depth': 9,
 'bagging_temperature': 0.7}

In [34]:
best_model=rcv.best_estimator_
import pickle 
import os

with open(os.path.join(os.getcwd(),'model.pkl'),'wb') as file_obj:
    pickle.dump(best_model,file_obj)

In [35]:
y_pred=best_model.predict(X_test)
print(r2_score(y_test,y_pred))

0.807069797136771
